# Packages

In [1]:
import numpy as np
import h5py as h5
import astropy.units as u
import astropy.constants as c


from IPython.display import clear_output
import time

from ipynb.fs.full.System_types import is_G_WD_fun, is_WD_G_fun
from ipynb.fs.full.Unstable_H_burning import hasHotComp_mask_WD_G_unstable, hasHotComp_mask_G_WD_unstable

# Importing data

In [2]:
path = '/STER/axelf/'  # Base directory for storing all files

# Define paths to different categories of data
path_data = path + 'simulations/'
path_energy = path_data + 'classic/'
path_twostage = path_data + 'twostage/'
path_unstable = path_data + 'unstable/'
path_stable = path_data + 'stable/'

# Define the path for observed data
path_obs = path + 'observations/'

In [3]:
# Load observed periods data
periods_obs = np.loadtxt(path_obs + '/periods.txt')

In [4]:
dataStables = []  # List to store data files for stable simulations
paramsStable = []  # List to store parameter sets (beta, gamma) for stable simulations

# Define gamma values and beta values range
gamma_values = [0.0, 0.5, 1.0]  # Specific gamma values to iterate over
beta_values = np.linspace(0.0, 1, 6)  # Beta values ranging from 0 to 1 in 6 steps

# Loop over different values of gamma and beta to load the corresponding simulation data
for g in gamma_values:  # Iterate over specified gamma values
    for b in beta_values:  # Iterate over beta values
                
        # Append the (beta, gamma) parameter pair to the paramsStable list
        paramsStable.append([b, g])
        
        # Construct the file path using formatted strings for clarity
        file_path = path_stable + "gamma_" + str(g) + "/beta_" + str(np.around(b,1)) + "/output_1/COMPAS_Output.h5"
        
        # Load the simulation data file (.h5 format) for the current beta and gamma
        dataStables.append(h5.File(file_path, 'r'))  # 'r' mode for read access


In [5]:
# Initialize a list to store energy data files
dataEnergys = []  # List to store data files for energy simulations

# Define the range of energy parameter values
energy_values = [-1, -0.5, 0, 0.5, 1]  # Specific energy parameter values

# Loop over specified energy parameter values to load corresponding simulation data
for energy in energy_values:
    # Construct the file path using formatted strings for clarity
    file_path = path_energy + "output_" + str(energy) +"_1/COMPAS_Output.h5"
    
    # Load the simulation data file (.h5 format) for the current energy value
    dataEnergys.append(h5.File(file_path, 'r'))  # 'r' mode for read access

In [6]:
# Initialize a list to store energy data files
dataUnstables = []  # List to store data files for energy simulations

# Loop over specified energy parameter values to load corresponding simulation data
for energy in energy_values:
    # Construct the file path using formatted strings for clarity
    file_path = path_unstable + "output_" + str(energy) +"_1/COMPAS_Output.h5"
    
    # Load the simulation data file (.h5 format) for the current energy value
    dataUnstables.append(h5.File(file_path, 'r'))  # 'r' mode for read access

In [7]:
# Load two-stage simulation data
data2Stage = h5.File(path_twostage + "output_1/COMPAS_Output.h5", 'r')  # Load two-stage data file

# Period SyNe

In [8]:
def timeWeight(Data):

    SWs = Data['BSE_Switch_Log']  # Access the switch log data
    
    # Extract relevant parameters from the switch log
    SemiMajor = SWs['SemiMajorAxis'][()] * u.Rsun  # Semi-major axis in solar radii
    Mass1 = SWs['Mass(1)'][()] * u.M_sun  # Mass of the first star in solar masses
    Mass2 = SWs['Mass(2)'][()] * u.M_sun  # Mass of the second star in solar masses
    Type1 = SWs['Stellar_Type(1)'][()]
    Type2 = SWs['Stellar_Type(2)'][()]    
    
    
    is_G_WD = is_G_WD_fun(Data)
        
    is_WD_G = is_WD_G_fun(Data)
    
    #############
    
    times = hasHotComp_mask_WD_G_unstable(Data, is_WD_G) # Myr
        
    periods = np.sqrt(4 * np.pi**2 * (SemiMajor[is_WD_G])**3 / (c.G * (Mass1[is_WD_G] + Mass2[is_WD_G])))

    
    #############
    
    # This part is often not necessary
    if (list(is_G_WD).count(True) > 0): 
            
        times_G_WD = hasHotComp_mask_G_WD_unstable(Data, is_G_WD)

        periods1 = np.sqrt(4 * np.pi**2 * (SemiMajor[is_G_WD])**3 / (c.G * (Mass1[is_G_WD] + Mass2[is_G_WD])))


        times = np.concatenate([times, times_G_WD])
        periods = np.concatenate([periods, periods1])
    
    return times, periods.to(u.day)  # Convert periods to days

# Results SyNe

In [9]:
start = time.time()

log_periods_obs = np.log10(periods_obs)

weight_2stage, periods_data2Stage = timeWeight(data2Stage)

log_periods_data2Stage = np.log10(periods_data2Stage.value)

clear_output(wait=True)

np.savetxt('../results/SyNe_results/twostage_results/2stage_log_periods', log_periods_data2Stage)
np.savetxt('../results/SyNe_results/twostage_results/2stage_weights', weight_2stage.value)

end = time.time()
print("It has been running for", np.round(end - start), "seconds.")
print("Twostage done")

It has been running for 192.0 seconds.
Twostage done


In [10]:
start = time.time()

log_periods_dataClassics = []
weight_Classics = []

for dataClassic in dataEnergys:

    weight_Classic, periods_dataClassic = timeWeight(dataClassic)
    
    weight_Classics.append(weight_Classic)

    log_periods_dataClassic = np.log10(periods_dataClassic.value)

    log_periods_dataClassics.append(log_periods_dataClassic)
    
    print("Classic subphase done")
    
clear_output(wait=True)

for i in range(len(energy_values)):

    np.savetxt('../results/SyNe_results/classic_results/classic_log_periods' + str(energy_values[i]), log_periods_dataClassics[i])
    np.savetxt('../results/SyNe_results/classic_results/classic_weights' + str(energy_values[i]), weight_Classics[i].value)

end = time.time()
print("It has been running for", np.round(end - start), "seconds.")
print("Classic done")

It has been running for 936.0 seconds.
Classic done


In [11]:
log_periods_dataUnstables = []
weight_Unstables = []

for dataUnstable in dataUnstables:
    
    weight_Unstable, periods_dataUnstable = timeWeight(dataUnstable)
    
    weight_Unstables.append(weight_Unstable)

    log_periods_dataUnstable = np.log10(periods_dataUnstable.value)

    log_periods_dataUnstables.append(log_periods_dataUnstable)

    print("Unstable subphase done")
    
for i in range(len(energy_values)):

    np.savetxt('../results/SyNe_results/unstable_results/unstable_log_periods' + str(energy_values[i]), log_periods_dataUnstables[i])
    np.savetxt('../results/SyNe_results/unstable_results/unstable_weights' + str(energy_values[i]), weight_Unstables[i].value)
    
clear_output(wait=True)


end = time.time()
print("It took", np.round(end - start), "seconds.")

print("Unstable done")

It took 1592.0 seconds.
Unstable done


In [12]:
log_periods_dataStables = []
weight_Stables = []

for dataStable in dataStables:
    
    weight_Stable, periods_dataStable = timeWeight(dataStable)
    
    weight_Stables.append(weight_Stable)

    log_periods_dataStable = np.log10(periods_dataStable.value)

    log_periods_dataStables.append(log_periods_dataStable)
    
    print("Stable subphase done")
    
clear_output(wait=True)

for g in range(len(gamma_values)):
    for b in range(len(beta_values)):
        np.savetxt('../results/SyNe_results/stable_results/stable_log_periods_' + str(gamma_values[g]) + '_' + str(np.round(beta_values[b],2)), log_periods_dataStables[3 * g + b])
        np.savetxt('../results/SyNe_results/stable_results/stable_weights_'     + str(gamma_values[g]) + '_' + str(np.round(beta_values[b],2)), weight_Stables[3 * g + b].value)

        end = time.time()
print("It has been running for", np.round(end - start), "seconds.")
print("Stable done")

It has been running for 4987.0 seconds.
Stable done
